In [4]:
def print_versions():
  import pkg_resources

  packages = ["lambeq","discopy","qiskit","qiskit-aer","qiskit-aer-gpu","qiskit-aer-gpu-cu11","pytket","pytket-qiskit","qiskit_ibm_runtime","cuQuantum-cu12","custatevec-cu12","cutensornet-cu12","cuquantum-python-cu12"]

# cuQuantum (cuStateVec)
  for package in packages:
      try:
          version = pkg_resources.get_distribution(package).version
          print(f"{package}: {version}")
      except pkg_resources.DistributionNotFound:
          print(f"{package}: Not installed")
      except Exception as e:
          print(f"Error: {e}")

print_versions()


from qiskit_aer import AerSimulator
print(AerSimulator().available_devices())
# AerSimulator().available_methods()

lambeq: 0.5.0
discopy: 1.2.2
qiskit: 1.4.5
qiskit-aer: 0.17.2
qiskit-aer-gpu: 0.15.1
qiskit-aer-gpu-cu11: Not installed
pytket: 2.11.0
pytket-qiskit: 0.66.0
qiskit_ibm_runtime: 0.44.0
cuQuantum-cu12: 25.11.1
custatevec-cu12: 1.11.0
cutensornet-cu12: 2.10.1
cuquantum-python-cu12: Not installed
('CPU', 'GPU')


In [ ]:
import qiskit_aer
import pkg_resources

print(f"Aer location: {qiskit_aer.__file__}")

# Check for multiple installations in the environment
aer_packages = [p for p in pkg_resources.working_set if 'qiskit-aer' in p.project_name]
print("Installed Aer packages found by pip:")
for p in aer_packages:
    print(f"- {p.project_name} ({p.version})")

In [ ]:
from pytket.extensions.qiskit.backends.aer import AerBackend

backend = AerBackend()
# Look at the underlying Qiskit object
qiskit_v = backend._qiskit_backend

print(f"Pytket is using: {qiskit_v.name}")
print(f"Available devices for this backend: {qiskit_v.available_devices()}")

In [ ]:
import os
import re
import sys
import json
import pickle
import contextlib
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict

from qiskit_aer import AerSimulator
# from pytket.extensions.qiskit import AerBackend # toks import veiks su qiskit v2.x
from pytket.extensions.qiskit.backends.aer import AerBackend


from lambeq.backend.grammar import Diagram, Id
from lambeq import ( BobcatParser, AtomicType, IQPAnsatz, Dataset,
                RemoveCupsRewriter, SimpleRewriteRule, Rewriter, UnifyCodomainRewriter,
                QuantumTrainer, TketModel, BinaryCrossEntropyLoss, SPSAOptimizer
                )


from tensorboard.backend.event_processing import event_accumulator

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile

# Create a big circuit to make the GPU work
qc = QuantumCircuit(29)
qc.h(range(20))
qc.measure_all()

sim = AerSimulator(method="statevector", device="GPU")
sim = AerSimulator(method="statevector", device="CPU")
print(sim.d)
print(sim.available_devices())
# This will throw an error if the GPU link is "fake"
job = sim.run(transpile(qc, sim), shots=10)
print("Simulation result successful:", job.result().get_counts())

In [ ]:
# 2. Check cuQuantum / cuStateVec
import cuquantum

try:
    print(f"✅ cuQuantum (cuStateVec) Version: {cuquantum.__version__}")
except Exception as e:
    print(f"❌ cuQuantum link error: {e}")

In [ ]:
# green@DESKTOP-5DCJBRK:/downloads$ export PATH=$PATH:/usr/local/cuda-13.1/bin
# green@DESKTOP-5DCJBRK:/downloads$ export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-13.1/lib64

In [ ]:
# custatevec-cu12
# cutensornet-cu12
# cuquantum-python-cu12

In [ ]:
# Compatable version
# "qiskit==1.4.5" "pytket-qiskit==0.66.0" "qiskit-ibm-runtime-0.44.0"

# pip install "qiskit==1.4.5" "pytket-qiskit==0.66.0" qiskit-aer-gpu lambeq discopy tensorboard

# Installed packages:
# 

# Successfully installed absl-py-2.3.1 blis-1.3.3 catalogue-2.0.10 click-8.3.1 cloudpathlib-0.23.0 confection-0.1.5 contourpy-1.3.3 cycler-0.12.1 cymem-2.0.13 discopy-1.2.2 filelock-3.20.3 fonttools-4.61.1 fsspec-2026.1.0 grpcio-1.76.0 h5py-3.15.1 hf-xet-1.2.0 huggingface-hub-0.36.0 kiwisolver-1.4.9 lambeq-0.5.0 markdown-3.10 matplotlib-3.10.8 murmurhash-1.0.15 nvidia-cublas-cu12-12.8.4.1 nvidia-cuda-cupti-cu12-12.8.90 nvidia-cuda-nvrtc-cu12-12.8.93 nvidia-cuda-runtime-cu12-12.8.90 nvidia-cudnn-cu12-9.10.2.21 nvidia-cufft-cu12-11.3.3.83 nvidia-cufile-cu12-1.13.1.3 nvidia-curand-cu12-10.3.9.90 nvidia-cusolver-cu12-11.7.3.90 nvidia-cusparse-cu12-12.5.8.93 nvidia-cusparselt-cu12-0.7.1 nvidia-nccl-cu12-2.27.5 nvidia-nvjitlink-cu12-12.8.93 nvidia-nvshmem-cu12-3.3.20 nvidia-nvtx-cu12-12.8.90 opt-einsum-3.4.0 pillow-12.1.0 preshed-3.0.12 protobuf-6.33.4 pyparsing-3.3.1 pyyaml-6.0.3 regex-2026.1.15 safetensors-0.7.0 smart-open-7.5.0 spacy-3.8.11 spacy-legacy-3.0.12 spacy-loggers-1.0.5 srsly-2.5.2 tensorboard-2.20.0 tensorboard-data-server-0.7.2 tensornetwork-0.4.6 thinc-8.3.10 tokenizers-0.22.2 torch-2.9.1 tqdm-4.67.1 transformers-4.57.6 triton-3.5.1 typer-slim-0.21.1 wasabi-1.1.3 weasel-0.4.3 werkzeug-3.1.5 wrapt-2.0.1

In [ ]:
# git config --global user.name "Green-255"
# git config --global user.email "naglis.kontautas04@gmail.com"